### Import packages

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import re
import time
import pandas as pd
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import TimeoutException

In [2]:
jumia_link = "https://www.jumia.com.eg/laptops/?page=1#catalog-listing"

links = []

# open chromedriver on your PC
def open_chromedriver():

    from selenium.webdriver import Chrome, ChromeOptions # Pictures are not displayed in page
                                                         # To download components quickly
        
    from selenium.webdriver.chrome.service import Service # to install The appropriate version for the user's browser
    from webdriver_manager.chrome import ChromeDriverManager 
    
    chromedriver_installer = Service(ChromeDriverManager().install())
    
    chromedriver_options = ChromeOptions()
    prefs = {'profile.managed_default_content_settings.images' : 2}# Pictures are not displayed in page
    chromedriver_options.add_experimental_option('prefs', prefs)
    
    web = Chrome(service = chromedriver_installer, chrome_options = chromedriver_options)
    
    return web


# scroll down all the page to load all data of item                                  
def scroll_down_all_the_way():
    
    from time import sleep
    
    global scroll_delay_s
    
    end = 0
    while end < web.execute_script('return document.body.scrollHeight'):
        start = end
        end = start + 1000
        web.execute_script(f'window.scrollTo({start}, {end})')
        sleep(2)

    
# get all links of laptops        
def get_laptops_links():
    global page_num
    for page in range(1, page_num+1):
        website_link = f"https://2b.com.eg/en/computers/laptops.html?p={page}"
        web.get(website_link)
        scroll_down_all_the_way()
        
        global soup
        soup = BeautifulSoup(web.page_source, 'html.parser')

        tag = soup.find("div", id="layer-product-list")
        global links

        info = tag.find("ol", class_="filterproducts products list items product-items").find_all("li")
        links = [link.a["href"] for link in info]
    
    return links

#get all links of laptops    
def get_laptops_links():
    global page_num
    for page in range(1, page_num+1):
        website_link = f"https://2b.com.eg/en/computers/laptops.html?p={page}"
        web.get(website_link) # open webdriver
        scroll_down_all_the_way()
        
        global soup
        soup = BeautifulSoup(web.page_source, 'html.parser') # send request

        tag = soup.find("div", id="layer-product-list")
        global links

        info = tag.find("ol", class_="filterproducts products list items product-items").find_all("li")
        links = [link.a["href"] for link in info]
    
    return links

# scrape data and append it in list of dictionary
def get_laptops_info(link):
    web.get(link)
    listing = {}
    listing["link"] = link
    time.sleep(3)
    global soup
    soup = BeautifulSoup(web.page_source, 'html.parser')
    info = soup.find("div", class_="product-info-main") # content of page
    
    if info:
        listing["title"] = info.find("div", class_="page-title-wrapper product").get_text(strip=True)
        listing["price"] = info.find("div", class_="price-box price-final_price").get_text(strip=True).split("Price")[-1]
        
        global l1
        global l2
        global l
        
        l1 = []
        l2 = []
        l = []
        
        desc = info.p.get_text(strip=True).split(":")
        for i in range(len(desc)):
            if i %2 == 0:
                l1.append(desc[i])

            if i %2 != 0:
                l2.append(desc[i])

        for defn, value in zip(l1, l2):
            l.append(f"{defn} : {value}")
            
        listing['more_info'] = "  ;  ".join(l) # Concate all features and spliting it by ";" 
        
    try:
        
        add_data = web.find_element(by="id", value="tab-label-additional").click()
    except:
        try:
            time.sleep(2)
            add_data = web.find_element(by="id", value="tab-label-additional").click()
        except:
            time.sleep(3)
            add_data = web.find_element(by="id", value="tab-label-additional").click()
    
    feature = soup.find("div", id="additional")
     # to save data in dectionary   
    if feature:                    
        global Keys
        global values

        tag = feature.find_all("tr")
        keys = [x.find_all("th")[0].get_text(strip=True) for x in tag]
        values = [x.find_all("td")[0].get_text(strip=True) for x in tag]
        dict_value = dict(zip(keys, values))
        listing.update(dict_value)

    print(link)
    print(f"{len(listing)} - ", listing)
    print("-" *100, "\n\n")
    

    return listing

In [3]:
web = open_chromedriver()
web.get(jumia_link)





====== WebDriver manager ======
====== WebDriver manager ======
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome
Current google-chrome version is 100.0.4896
Get LATEST chromedriver version for 100.0.4896 google-chrome
Driver [C:\Users\Mora\.wdm\drivers\chromedriver\win32\100.0.4896.60\chromedriver.exe] found in cache
Driver [C:\Users\Mora\.wdm\drivers\chromedriver\win32\100.0.4896.60\chromedriver.exe] found in cache
C:\Users\Mora\AppData\Local\Temp/ipykernel_14212/3277877363.py:17: DeprecationWarning: use options instead of chrome_options
  web = Chrome(service = chromedriver_installer, chrome_options = chromedriver_options)
C:\Users\Mora\AppData\Local\Temp/ipykernel_14212/3277877363.py:17: DeprecationWarning: use options instead of chrome_options
  web = Chrome(service = chromedriver_installer, chrome_options = chromedriver_options)


In [4]:
csv = pd.read_csv(r"E:\other\Epsilon_Ai\Final Project\jumia.csv")
df = pd.DataFrame(csv)
df

,Unnamed: 0,info,price,link,prod_name,processor,screen_size,storeg_size,ram_size,gpu,os,store
0,0,Lenovo V15 ADA Laptop - AMD 3020E - 4GB RAM - ...,"EGP 5,555.00",https://www.jumia.com.eg/lenovo-v15-ada-laptop...,Lenovo V15 ADA Laptop,AMD 3020E,"15.6"" HD",1TB HDD,4GB RAM,AMD Radeon Graphics,DOS,jumia
1,1,DELL G3 15-3500 Gaming Laptop - Intel Core I5-...,"EGP 16,444.00",https://www.jumia.com.eg/g3-15-3500-gaming-lap...,DELL G3 15-3500 Gaming Laptop,Intel Core I5-10300H,4GB GPU,256GB SSD + 1TB HDD,8GB RAM,15.6-inch Full HD,Ubuntu,jumia
2,2,Lenovo IdeaPad 3-15IML05 Laptop - Intel Core I...,"EGP 8,149.00",https://www.jumia.com.eg/ideapad-3-15iml05-lap...,Lenovo IdeaPad 3-15IML05 Laptop,Intel Core I3,2GB GPU,1TB HDD,4GB RAM,15.6-inch FHD,DOS,jumia
3,3,DELL G15 5511 Gaming Laptop - Intel Core I7-11...,"EGP 22,999.00",https://www.jumia.com.eg/g15-5511-gaming-lapto...,DELL G15 5511 Gaming Laptop,Intel Core I7-11800H,15.6 Inch FHD 120Hz Display,512GB SSD,16GB RAM,NVIDIA GeForce RTX 3050 4GB,Ubuntu,jumia
4,4,DELL Vostro 3510 - Intel Core I7 - 16GB RAM - ...,"EGP 17,499.00",https://www.jumia.com.eg/dell-vostro-3510-inte...,DELL Vostro 3510,Intel Core I7,15.6-inch FHD,1TB HDD,16GB RAM,512GB SSD,2GB GPU,jumia
...,...,...,...,...,...,...,...,...,...,...,...,...
299,427,DELL G3 15-3590 Gaming Laptop - Intel Core i7-...,"EGP 16,777.00",https://www.jumia.com.eg/dell-g3-15-3590-gamin...,DELL G3 15-3590 Gaming Laptop,Intel Core i7-9750H,4GB GTX 1650 GPU,256GB SSD + 1TB HDD,16GB RAM,15.6-inch FHD,Windows 10,jumia
300,428,DELL G3 15-3590 Gaming Laptop - Intel Core i7-...,"EGP 18,799.00",https://www.jumia.com.eg/dell-g3-15-3590-gamin...,DELL G3 15-3590 Gaming Laptop,Intel Core i7-9750H,6GB GTX 1660TI GPU,256GB SSD + 1TB HDD,16GB RAM,15.6-inch FHD,Windows 10,jumia
301,429,DELL G5 15-5590 Laptop - Intel Core I7-9750H -...,"EGP 26,999.00",https://www.jumia.com.eg/dell-g5-15-5590-lapto...,DELL G5 15-5590 Laptop,Intel Core I7-9750H,8GB RTX 2070 GPU,512GB SSD + 1TB HDD,16GB RAM,15.6-inch FHD,Windows 10,jumia
302,430,DELL Inspiron 15-3593 Laptop - Intel Core i7-1...,"EGP 11,499.00",https://www.jumia.com.eg/dell-inspiron-15-3593...,DELL Inspiron 15-3593 Laptop,Intel Core i7-1065G7,4GB MX230 GPU,1TB HDD,8GB RAM,15.6-inch FHD,Ubuntu,jumia
